In [ ]:
import numpy as np              # basic arrays, vectors, matrices
import numpy.linalg as la       # linear algebra functions
import matplotlib.pyplot as plt # plotting
from scipy.linalg import lu, lu_factor, lu_solve
from numpy import array, zeros, fabs, linalg
%matplotlib inline

In [ ]:
Test = np.array([[1, 2, 3, 5, -2], [0, 6, -3, 1, -4], [2, -8, -8, 2, 5], [3, 5, -3, 7, -1], [7, 2, 3, 5, -2]])
n = Test.shape[0]
print (Test)

y = np.array( [3., 5, 2, 3, 1] )

TE = np.array([[2],
              [3],
              [0],
              [9],
              [-2]])
print(TE)
i = np.argmax(Test[:,0])    #Returns the indices of the maximum values along an axis.
print(i)
A_bar = np.vstack([Test[i,:], Test[:i,:], Test[(i+1):,:]])   #Stack arrays in sequence vertically (row wise).
print(A_bar)

A_bar11 = A_bar[0,0]    #Eintrag (0,0)
#print(A_bar11)
A_bar12 = A_bar[0,1:]   #Eintrag erste Reihe ohne (0,0)
#print(A_bar12)
A_bar21 = A_bar[1:,0]   #Eintrag erste Spalte ohne (0,0)
#print(A_bar21)
A_bar22 = A_bar[1:,1:]  #Eintrag übrige Matrix 
#print(A_bar22)

S22 = A_bar22 - np.dot(A_bar21, A_bar12) / A_bar11   #Dot product of two arrays
print(S22)

L22 = S22

U22 = S22
P22 = S22
    
L11 = 1
U11 = A_bar11
#print(U11)
L12 = np.zeros(n-1)
U12 = A_bar12.copy()

L21 = np.dot(P22, A_bar21) / A_bar11

LX = L21.reshape(4,1)
print("LX")
print(LX)

print("L11:")
print(L11)
print("L12:")
print(L21)
print("L21:")
print(L21)
print("L22:")
print(L22)


U21 = np.zeros(n-1)

L1 = np.block([L11, L12])
print("L:")
print(L1)
L2 = np.block([LX, L22])
print("L2")
print(L2)

#print(L2)
L = np.block([[L1], [L2]])
print(L)
#Ä = np.block([L21, L22])

In [ ]:
    P = np.block([
        [np.zeros((1, i-1)), 1,                  np.zeros((1, n-i))],
        [P22[:,:(i-1)],      np.zeros((n-1, 1)), P22[:,i:]]
    ])

In [ ]:
n = 50
x = np.linspace( -1, 1, n )
A = np.vander( x, increasing=True )
print(A)

In [ ]:
def lup_decomp(A):
    n = A.shape[0]
    if n == 1:
        L = np.array([[1]])
        U = A.copy()
#        P = np.array([[1]])
        return (L, U)

    i = np.argmax(A[:,0])
    A_bar = np.vstack([A[i,:], A[:i,:], A[(i+1):,:]])

    A_bar11 = A_bar[0,0]
    A_bar12 = A_bar[0,1:]
    A_bar21 = A_bar[1:,0]
    A_bar22 = A_bar[1:,1:]

    S22 = A_bar22 - np.dot(A_bar21, A_bar12) / A_bar11

    L22 = S22
    U22 = S22
    P22 = S22

    L11 = 1
    U11 = A_bar11

    L12 = np.zeros(n-1)
    U12 = A_bar12.copy()

    L21 = np.dot(P22, A_bar21) / A_bar11
    U21 = np.zeros(n-1)

    UX = U21.reshape(4,1)
    LX = L21.reshape(4,1)
    
    L1 = np.block([L11, L12])
    L2 = np.block([LX, L22])
    U1 = np.block([U11, U12])
    U2 = np.block([UX, U22])

    L = np.block([[L1], [L2]])
    U = np.block([[U1], [U2]])
#    P = np.block([
#        [np.zeros((1, i-1)), 1,                  np.zeros((1, n-i))],
#        [P22[:,:(i-1)],      np.zeros((n-1, 1)), P22[:,i:]]
#    ])
    return (L, U)

#print(A)
#print(L)
#print(U)
#print(P)

In [ ]:
L, U = lup_decomp(Test)

print(L)
print(U)

In [ ]:
def fwd_subs( L, b ):
    """Solve the linear system Ly = b with lower triangular matrix L by forward substitution."""
    
    n = L.shape[0]
    x = np.zeros(n)
    for i in range(n):
        Sum = b[i]
        for j in range(i):
            Sum -= L[i,j] * x[j]
        x[i] = Sum / L[i,i]
    return x

In [ ]:
x = fwd_subs(Test, y)
print(x)

In [ ]:
def abc(a):
    n = len(a)
    for k in range(0, n-1):
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lam = a [i,k]/a[k,k]
                a[i, k+1:n] = a[i, k+1:n] - lam*a[k, k+1:n]
                a[i,k] = lam
    return a

In [ ]:
A5 = np.array([[1.0, 4.0, 1.0], [1.0, 6.0, -1.0], [2.0, -1.0, 2.0]])

BB = abc(A5)
print(A5)
print(BB)

In [213]:
AT = np.array([[2, 4, 5, 6], [-1, 2, 8.8, 1], [3, 8, 3, -3], [5, 2, 1.5, 6.4]])
Test3 = np.array([[1., -3., 22.], [3., 5., -6.], [4., 235., 7.]])

def LU(A, demoMode=False):
    n=A.shape[0]
    perm = array(range(n))
    P = np.eye(n)
    for k in range(n-1):
        if demoMode: print(f"{k=}")

        # Find the pivot element in column k:
        pivot_row = k
        abs_u_ik_max = abs(A[perm[k],k])
        for row in range(k+1, n):
            abs_u_ik = abs(A[perm[row],k])
            if abs_u_ik > abs_u_ik_max:
                pivot_row = row
                abs_u_ik_max = abs_u_ik
        if pivot_row > k: # "swap"
            if demoMode: print(f"Swap row {k} with row {pivot_row}")
            (perm[k], perm[pivot_row]) = (perm[pivot_row], perm[k])
        else:
            if demoMode: print(f"No row sawp needed.")
        if demoMode: 
            print(f"No row swap needed.")
            print(f"permuted A is:")
            for row in range(n):
                stack = np.vstack((perm[row]))
                print(A[perm[row],:])
                    
    U=np.copy(A)
    L=np.eye(n)
    for i in range(n):
        p=U[i,i]
        for j in range(i+1,n):
            L[j,i]=U[j,i]/p
            U[j]=U[j]-L[j,i]*U[i]
    return L, U, P, perm

In [214]:
AT

zz = LU(Test3)
zz
#perm gibt an, welche die Reihenfolge der Zeilen aus A
#[2, 1, 0] -> Zeile 2 an oberstere Stelle, Zeile 0 an unterster

(array([[ 1.        ,  0.        ,  0.        ],
        [ 3.        ,  1.        ,  0.        ],
        [ 4.        , 17.64285714,  1.        ]]),
 array([[ 1.00000000e+00, -3.00000000e+00,  2.20000000e+01],
        [ 0.00000000e+00,  1.40000000e+01, -7.20000000e+01],
        [ 0.00000000e+00,  0.00000000e+00,  1.18928571e+03]]),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([2, 1, 0]))

In [242]:
n = 3
P = np.eye(n)

perm = np.array([2, 1, 0])
stelle_max = perm[max(perm)]
stelle_mitte = perm[max(perm)-1]
stelle_unten = perm[max(perm)-2]
P2 = P[2]

stelle_max

0

In [197]:
AT

array([[ 2. ,  4. ,  5. ,  6. ],
       [-1. ,  2. ,  8.8,  1. ],
       [ 3. ,  8. ,  3. , -3. ],
       [ 5. ,  2. ,  1.5,  6.4]])

In [209]:
Test3

array([[  1.,  -3.,  22.],
       [  3.,   5.,  -6.],
       [  4., 235.,   7.]])

In [ ]:
from numpy import array, zeros_like

def plu(A, demoMode=False):
    """Compute the Doolittle PA=LU factorization of A —
    but with the permutation recorded as permutation vector, not as the permutation matrix P.
    Sums like $\sum_{s=1}^{k-1} l_{k,s} u_{s,j}$ are done as matrix products;
    in the above case, row matrix L[k, 1:k-1] by column matrix U[1:k-1,j] gives the sum for a give j,
    and row matrix L[k, 1:k-1] by matrix U[1:k-1,k:n] gives the relevant row vector.
    
    NOT WORKING YET!
    """
    n = len(A)  # len() gives the number of rows in a 2D array.
    perm = array(range(n))
    print(perm)
    print(f"Perm:{perm}")
    # Initialize U as the zero matrix;
    # correct below the main diagonal, with the other entries to be computed below.
    U = zeros_like(A)
    # Also, initialize L as the zero matrix;
    # the 1's will also be filled in as we go.
    L = zeros_like(A)
    for k in range(n-1):
        if demoMode: print(f"{k=}")

        # Find the pivot element in column k:
        pivot_row = k
        abs_u_ik_max = abs(A[perm[k],k])
        for row in range(k+1, n):
            abs_u_ik = abs(A[perm[row],k])
            if abs_u_ik > abs_u_ik_max:
                pivot_row = row
                abs_u_ik_max = abs_u_ik
        if pivot_row > k: # "swap"
            if demoMode: print(f"Swap row {k} with row {pivot_row}")
            (perm[k], perm[pivot_row]) = (perm[pivot_row], perm[k])
        else:
            if demoMode: print(f"No row swap needed.")
        U[k,k:] = A[perm[k],k:] - L[perm[k],:k] @ U[:k,k:]

        L[perm[k],k] = 1. 
        for row in range(k+1,n):
            L[perm[row],k] = ( A[perm[row],k] - L[perm[row],:k] @ U[:k,k] ) / U[k,k]
        if demoMode:
            print(f"permuted A is:")
            for row in range(n)
:
                print(A[perm[row],:])
            print(f"intermediate U is\n{U}")
            print(f"intermediate L is\n{L}")
            print(f"perm={perm}")
    # The last row (index "-1") is special: nothing to do for L except put in the 1 on the "permuted main diagonal"
    U[n-1,n-1] = A[perm[n-1],n-1] - sum(L[perm[n-1],:n-1]*U[:n-1,n-1])
    L[perm[n-1],n-1] = 1. 
    if demoMode:
        print(f"After the final step, k={n-1}")
        print(f"U=\n{U}")
    return (L, U, perm)

In [163]:
sdfa = np.array([[2, 4, 5, 6], [-1, 2, 8.8, 1], [3, 8, 3, -3], [5, 2, 1.5, 6.4]])
sdfa

array([[ 2. ,  4. ,  5. ,  6. ],
       [-1. ,  2. ,  8.8,  1. ],
       [ 3. ,  8. ,  3. , -3. ],
       [ 5. ,  2. ,  1.5,  6.4]])